In [158]:
import requests
import pandas as pd
import numpy as np
import lxml.html as lh

The url to the wikipage from where the data has to be picked is saved in the variable website_url
The contents of the wikipage is fetched and based on the tag <tr> on the html page and the column count which is 3 the html is parsed to create a dataframe

In [159]:
website_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = requests.get(website_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')    

In [160]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [161]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [167]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

#The column headers are renamed
df.rename(columns={'Postcode':'PostalCode','Neighbourhood\n':'Neighborhood'}, inplace=True)
df['Neighborhood'] = df['Neighborhood'].map(lambda x: x.rstrip('\n'))

#rows having Borough column value as Not assigned is dropped
indexNames = df[ df['Borough'] == 'Not assigned' ].index
df.drop(indexNames , inplace=True)

#rows having Neighbourhood column value as Not assigned is replaced by Borough column value
df['Neighborhood'] = np.where(df.Neighborhood!='Not assigned', df.Neighborhood, df.Borough)

df.shape

(212, 3)

In [168]:
#where multiple rows exist for a postal code the rows are combined and the values in the Neighborhood column in concatenated using a ,
df.set_index(['PostalCode', 'Borough'], inplace=True)
result = df.groupby(level=['PostalCode','Borough'], sort=False).agg( ','.join)
print(result)

                                                                  Neighborhood
PostalCode Borough                                                            
M3A        North York                                                Parkwoods
M4A        North York                                         Victoria Village
M5A        Downtown Toronto                           Harbourfront,Regent Park
M6A        North York                          Lawrence Heights,Lawrence Manor
M7A        Queen's Park                                           Queen's Park
M9A        Etobicoke                                          Islington Avenue
M1B        Scarborough                                           Rouge,Malvern
M3B        North York                                          Don Mills North
M4B        East York                            Woodbine Gardens,Parkview Hill
M5B        Downtown Toronto                            Ryerson,Garden District
M6B        North York                               

In [171]:
#there are 103 unique postal code values in the dataframe
result.shape

(103, 1)